# Methods to try for comparing Arro to training data

1. Probit model using resampling with the training data - done
2. Probit model with SMOTE (do this in the end, might be tricky without snowpark (limitation))
3. XGBoost or any other tree boosting method for classification between both datasets
    3.1 Evaluate using AUC, confusion matrix and any other suitable metrics
    3.2 Expectations : if accuracy is low then similarity is more?

Arro customers : 3862
Train data : 1,048,271

In [437]:
1048271/3862

271.43215950284826

In [2]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import month,year,col,sum
from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import LongType, DecimalType

# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [3]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit
import scipy.stats as ss
import numpy as np
import pandas as pd

In [4]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))

User                        : VEDAANTIBALIGA
Role                        : "PUBLIC"
Snowflake version           : 7.36.0
Snowpark for Python version : 1.5.1
Warehouse                   : "ANALYSIS_XS"


#### Get external and Arro data from snowflake

In [404]:
# list of features
features = ['NUM_INQ_3M', 'NUM_INQ_24M', 'PCT_UTIL_INQ_12M_TO_INQ_24M',
       'NUM_UTIL_INQ_6M', 'NUM_NON_UTIL_INQ_6M', 'AGE_OLDEST_BANKCARD_TRADE',
       'AGE_NEWEST_TRADE', 'AGE_NEWEST_BANKCARD_TRADE',
       'AGE_NEWEST_DEP_STORE_TRADE', 'NUM_OPEN_CREDIT_UNION_TRADES',
       'NUM_OPEN_MORTGAGE_TRADES', 'TOT_BAL_OPEN_CREDIT_UNION_TRADES',
       'TOT_BAL_OPEN_DEP_STORE_TRADES', 'TOT_BAL_OPEN_INSTALLMENT_TRADES',
       'TOT_BAL_OPEN_RETAIL_TRADES', 'NUM_MORTGAGE_TRADES',
       'TOT_HIGH_CREDIT_OPEN_BANKCARD', 'TOT_LOAN_AMT_OPEN_INSTALLMENT',
       'TOT_HIGH_CREDIT_OPEN_REVOLVING', 'NUM_TRADES_PAST_DUE_BALANCE',
       'NUM_REVOLVING_TRADES_PAST_DUE_BAL', 'TOT_PAST_DUE_BAL',
       'TOT_PAST_DUE_BAL_RETAIL', 'NUM_30D_PAST_DUE_24M',
       'NUM_60PLUSD_PAST_DUE_24M', 'NUM_60PLUSD_PAST_DUE_24M_REVOLVING',
       'NUM_TRADES_WORST_6M_30D_PAST_DUE', 'NUM_TRADES_30PLUSD_PAST_DUE',
       'NUM_OPEN_BANKCARD_TRADES_UTIL_GTE_75PCT', 'NUM_UNPAID_COLLECTIONS',
       'TOTAL_UNPAID_COLLECTIONS_BAL_12M', 'TOTAL_UNPAID_COLLECTIONS_BAL_24M',
       'DISCHARGED_BANKRUPTY', 'PCT_BANKCARD_TO_TOT_TRADES',
       'PCT_REVOLVING_TO_TOT_TRADES', 'PCT_TRADES_OPENED_12M_TO_TOT_TRADES',
       'TOT_BAL_REVOLVING_TRADES', 'TOT_UTIL_BANKCARD_TRADES',
       'PCT_BAL_PAST_DUE_ALL_TRADES',
       'PCT_GOOD_REVOLVING_TRADES_TO_TOT_REVOLVING_TRADES',
       'PCT_TRADES_GOOD_6M_TO_TOT_TRADES_6M',
       'PCT_REVOLVING_TRADES_GOOD_6M_TO_REVOLVING_TRADES_6M',
       'NUM_TRADES_WORST_EVER_60PLUSD_PAST_DUE',
       'PCT_TRADES_WORST_EVER_60PLUSD_PAST_DUE_TO_TOT_TRADES',
       'AVG_SALARY_INCOME_TXN', 'AVG_UTILITIES_EXPENSE_TXN',
       'COUNT_ASSET_DEPOSIT_TXN', 'COUNT_NEG_BAL_OCC_0_180',
       'COUNT_NEG_BAL_OCC_0_30', 'COUNT_RENT_EXPENSE_TXN_0_365',
       'COUNT_TELECOM_EXPENSE_TXN_0_30', 'SM_CASH_OUT_DEBT_SER',
       'SUM_CASH_OUTFLOW_TRANSFERS', 'SUM_INSURANCE_EXPENSE_TXN',
       'SUM_SALARY_INCOME_TXN_0_365', 'SUM_UTILITIES_EXPENSE_TXN',
       'TOTAL_BALANCE']

In [438]:
external = session.sql("""select a.*, uniform(1,10,random()) as uniform 
            FROM EQUIFAX_PRESALE.CREDIT_INCOME.JUNE2019_UPDATED a
            where VANTAGE_SCORE > 660 and uniform = 1
            union
            select a.*, uniform(1,10,random()) as uniform
            FROM EQUIFAX_PRESALE.CREDIT_INCOME.JUNE2019_UPDATED a
            where VANTAGE_SCORE <= 660 
  limit 3862
""")

In [439]:
external = external[features]

In [433]:
arro = session.sql("""select * from credit.application.arro_risk_model_1_summary
                      where user_id in (select user_id from CREDIT.APPLICATION.APPLICATION_SUMMARY where testing_stage = 'Rollout' AND application_recency = 1 AND lower(APPLICATION_STATUS) = 'approved')  """)

In [16]:
arro_new = session.sql("""select * from credit.application.arro_risk_model_1_summary
                      where user_id in (select user_id from CREDIT.APPLICATION.APPLICATION_SUMMARY 
                      where testing_stage = 'Rollout' AND application_recency = 1 AND lower(APPLICATION_STATUS) = 'approved')  
                       and APPLICATION_START_TS > '2023-10-04' """)

In [17]:
print(arro_new.count())

145


In [440]:
arro = arro[features]

In [441]:
print(external.count())
print(arro.count())

3862
3862


Pre-process and clean data

In [442]:
# check shape and null values
print("Arro count: {} Arro null values: {}, null is {}% of data".format(arro.count(), arro.count() - arro.dropna().count(), round((arro.count() - arro.dropna().count())/arro.count() * 100)))
print("Train count: {} Train null values: {}, null is {}% of data".format(external.count(), external.count() - external.dropna().count(), round((external.count() - external.dropna().count())/external.count() * 100)))

Arro count: 3862 Arro null values: 3527, null is 91% of data
Train count: 3862 Train null values: 0, null is 0% of data


In [443]:
# convert external_data and arro_data to pandas dataframes
external_data = external.to_pandas()
arro_data = arro.to_pandas()

In [445]:
arro_data

,NUM_INQ_3M,NUM_INQ_24M,PCT_UTIL_INQ_12M_TO_INQ_24M,NUM_UTIL_INQ_6M,NUM_NON_UTIL_INQ_6M,AGE_OLDEST_BANKCARD_TRADE,AGE_NEWEST_TRADE,AGE_NEWEST_BANKCARD_TRADE,AGE_NEWEST_DEP_STORE_TRADE,NUM_OPEN_CREDIT_UNION_TRADES,...,COUNT_NEG_BAL_OCC_0_180,COUNT_NEG_BAL_OCC_0_30,COUNT_RENT_EXPENSE_TXN_0_365,COUNT_TELECOM_EXPENSE_TXN_0_30,SM_CASH_OUT_DEBT_SER,SUM_CASH_OUTFLOW_TRANSFERS,SUM_INSURANCE_EXPENSE_TXN,SUM_SALARY_INCOME_TXN_0_365,SUM_UTILITIES_EXPENSE_TXN,TOTAL_BALANCE
0,0.0,2.0,0.0,98.0,1.0,3.0,2.0,3.0,9998.0,98.0,...,0.0,0.0,NaN,NaN,13414.30,15968.44,NaN,26612.40,261.18,0.10
1,2.0,2.0,0.0,98.0,2.0,24.0,1.0,1.0,9998.0,98.0,...,0.0,0.0,NaN,NaN,21330.65,20283.39,100.00,33189.48,876.27,32.99
2,1.0,2.0,0.0,98.0,1.0,9998.0,15.0,9998.0,9998.0,0.0,...,0.0,0.0,NaN,1.0,NaN,6235.56,NaN,31430.64,392.40,0.00
3,1.0,5.0,0.0,98.0,1.0,27.0,10.0,27.0,9998.0,98.0,...,56.0,10.0,NaN,2.0,25889.75,33216.82,1239.36,66888.12,6027.51,-33.46
4,2.0,3.0,0.0,98.0,2.0,73.0,2.0,2.0,58.0,98.0,...,20.0,0.0,NaN,NaN,5531.10,6643.63,NaN,14320.92,NaN,94.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,6.0,31.0,0.0,98.0,11.0,106.0,3.0,5.0,8.0,98.0,...,36.0,6.0,NaN,NaN,17445.15,43343.48,NaN,36090.60,NaN,0.76
3858,5.0,26.0,0.0,98.0,5.0,9997.0,7.0,9997.0,9997.0,98.0,...,95.0,26.0,NaN,NaN,25823.90,16195.30,NaN,91.68,284.34,0.50
3859,0.0,2.0,0.0,98.0,0.0,90.0,21.0,26.0,9998.0,98.0,...,48.0,8.0,3.0,NaN,18245.10,10422.14,NaN,45121.68,318.27,121.35
3860,1.0,5.0,0.0,98.0,1.0,27.0,5.0,20.0,9997.0,98.0,...,84.0,10.0,NaN,1.0,3280.05,19880.41,NaN,41194.08,1461.00,1649.53


### Impute NA values with median

In [446]:
arro_data_copy = arro_data.copy()
external_data_copy = external_data.copy()

In [447]:
arro_data = arro_data_copy.copy()
external_data = external_data_copy.copy()

In [448]:
# get the file with the null mapping
null_mapping = pd.read_csv('data/feature_null_mapping.csv')
null_mapping = null_mapping[['Column Name', 'NULL Values']].reset_index()[['Column Name', 'NULL Values']]
null_mapping['Column Name'] = null_mapping['Column Name'].str.upper()
null_mapping = null_mapping[null_mapping['Column Name'].isin(arro_data.columns)]
null_mapping.set_index('Column Name', inplace=True)

In [449]:
null_mapping.to_csv('data/new_null_mapping.csv')

In [450]:
null_mapping = pd.read_csv('data/new_null_mapping.csv',index_col=0)

In [451]:
for index, row in null_mapping.iterrows():
    null_mapping.at[index, 'NULL Values'] = [float(x) for x in row['NULL Values'].split(',')]

In [452]:
for i in arro_data.columns:
    val = null_mapping.loc[i]['NULL Values']
    arro_data[i] = arro_data[i].apply(lambda x: np.nan if x in val else x)
    external_data[i] = external_data[i].apply(lambda x: np.nan if x in val else x)

In [453]:
arro_data

,NUM_INQ_3M,NUM_INQ_24M,PCT_UTIL_INQ_12M_TO_INQ_24M,NUM_UTIL_INQ_6M,NUM_NON_UTIL_INQ_6M,AGE_OLDEST_BANKCARD_TRADE,AGE_NEWEST_TRADE,AGE_NEWEST_BANKCARD_TRADE,AGE_NEWEST_DEP_STORE_TRADE,NUM_OPEN_CREDIT_UNION_TRADES,...,COUNT_NEG_BAL_OCC_0_180,COUNT_NEG_BAL_OCC_0_30,COUNT_RENT_EXPENSE_TXN_0_365,COUNT_TELECOM_EXPENSE_TXN_0_30,SM_CASH_OUT_DEBT_SER,SUM_CASH_OUTFLOW_TRANSFERS,SUM_INSURANCE_EXPENSE_TXN,SUM_SALARY_INCOME_TXN_0_365,SUM_UTILITIES_EXPENSE_TXN,TOTAL_BALANCE
0,0.0,2.0,0.0,NaN,1.0,3.0,2.0,3.0,NaN,NaN,...,0.0,0.0,NaN,NaN,13414.30,15968.44,NaN,26612.40,261.18,0.10
1,2.0,2.0,0.0,NaN,2.0,24.0,1.0,1.0,NaN,NaN,...,0.0,0.0,NaN,NaN,21330.65,20283.39,100.00,33189.48,876.27,32.99
2,1.0,2.0,0.0,NaN,1.0,NaN,15.0,NaN,NaN,0.0,...,0.0,0.0,NaN,1.0,NaN,6235.56,NaN,31430.64,392.40,0.00
3,1.0,5.0,0.0,NaN,1.0,27.0,10.0,27.0,NaN,NaN,...,56.0,10.0,NaN,2.0,25889.75,33216.82,1239.36,66888.12,6027.51,-33.46
4,2.0,3.0,0.0,NaN,2.0,73.0,2.0,2.0,58.0,NaN,...,20.0,0.0,NaN,NaN,5531.10,6643.63,NaN,14320.92,NaN,94.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,6.0,31.0,0.0,NaN,11.0,106.0,3.0,5.0,8.0,NaN,...,36.0,6.0,NaN,NaN,17445.15,43343.48,NaN,36090.60,NaN,0.76
3858,5.0,26.0,0.0,NaN,5.0,NaN,7.0,NaN,NaN,NaN,...,95.0,26.0,NaN,NaN,25823.90,16195.30,NaN,91.68,284.34,0.50
3859,0.0,2.0,0.0,NaN,0.0,90.0,21.0,26.0,NaN,NaN,...,48.0,8.0,3.0,NaN,18245.10,10422.14,NaN,45121.68,318.27,121.35
3860,1.0,5.0,0.0,NaN,1.0,27.0,5.0,20.0,NaN,NaN,...,84.0,10.0,NaN,1.0,3280.05,19880.41,NaN,41194.08,1461.00,1649.53


Check all columns

In [455]:
for i in arro_data.columns:
    # write all this in a file
    with open('data_values.txt', 'a') as f:
        f.write('Arro Data '+i+ '\n')
        f.write(str(arro_data[i].value_counts()) + '\n')
        f.write('Null values '+str(arro_data[i].isnull().sum()) + '\n')
        f.write('External Data '+i+ '\n')
        f.write(str(external_data[i].value_counts()) + '\n')
        f.write('Null values '+str(external_data[i].isnull().sum()) + '\n')
f.close()

In [456]:
len(arro_data.columns)

57

In [457]:
original_columns = arro_data.columns

In [458]:
# Remove columns from both arro_data and external_data if 60% of either column data is null
for i in arro_data.columns:
    if (arro_data[i].isnull().sum() > 0.6 * len(arro_data)) or (external_data[i].isnull().sum() > 0.6 * len(external_data)):
        arro_data.drop(i, axis=1, inplace=True)
        external_data.drop(i, axis=1, inplace=True)

In [459]:
# see which columns were removed
for i in original_columns:
    if i not in arro_data.columns:
        print(i)

NUM_UTIL_INQ_6M
AGE_NEWEST_DEP_STORE_TRADE
NUM_OPEN_CREDIT_UNION_TRADES
NUM_OPEN_MORTGAGE_TRADES
TOT_BAL_OPEN_CREDIT_UNION_TRADES
TOT_BAL_OPEN_DEP_STORE_TRADES
TOT_BAL_OPEN_INSTALLMENT_TRADES
TOT_BAL_OPEN_RETAIL_TRADES
TOTAL_UNPAID_COLLECTIONS_BAL_12M
TOTAL_UNPAID_COLLECTIONS_BAL_24M
COUNT_RENT_EXPENSE_TXN_0_365


In [460]:
# fill arro_data with median values
for i in arro_data.columns:
    arro_data[i].fillna(arro_data[i].median(), inplace=True)
for i in external_data.columns:
    external_data[i].fillna(external_data[i].median(), inplace=True)

In [461]:
len(arro_data.columns)

46

Arro data : 
PCT_UTIL_INQ_12M_TO_INQ_24M - remove, why is this not there?
NUM_MORTGAGE_TRADES, - dont remove
TOT_HIGH_CREDIT_OPEN_BANKCARD, - this is credit line, dont remove
TOT_PAST_DUE_BAL_RETAIL,  - dont remove
NUM_TRADES_WORST_6M_30D_PAST_DUE, - dont remove
NUM_TRADES_30PLUSD_PAST_DUE, - dont remove
NUM_OPEN_BANKCARD_TRADES_UTIL_GTE_75PCT, - dont remove
NUM_UNPAID_COLLECTIONS, - dont remove
DISCHARGED_BANKRUPTY, - this is right, binary variable
COUNT_NEG_BAL_OCC_0_30, - dont remove
COUNT_TELECOM_EXPENSE_TXN_0_30, - dont remove
SUM_INSURANCE_EXPENSE_TXN,  - dont remove

In [462]:
arro_data.drop('PCT_UTIL_INQ_12M_TO_INQ_24M', axis=1, inplace=True)
external_data.drop('PCT_UTIL_INQ_12M_TO_INQ_24M', axis=1, inplace=True)

In [463]:
arro_data.to_csv('data/arro_data.csv',index=False)